In [1]:
import os, sys
import numpy as np
current_dir = os.getcwd()
DATA_HOME_DIR = current_dir+'/data/ml-small/'

In [2]:
import importlib
import utils; importlib.reload(utils)
from utils import *
from __future__ import division, print_function
%matplotlib inline

Using cuDNN version 7003 on context None
Mapped name None to device cuda0: GeForce GTX 1080 Ti (0000:01:00.0)
Using Theano backend.


In [3]:
path = DATA_HOME_DIR
model_path = path+'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size = 64

In [4]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [5]:
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()

In [6]:
users = ratings.userId.unique()
movies = ratings.movieId.unique()

In [7]:
len (ratings)

100004

In [8]:
userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [9]:
ratings.movieId = ratings.movieId.apply(lambda x: movieid2idx[x])
ratings.userId = ratings.userId.apply(lambda x: userid2idx[x])

In [10]:
ratings.head()

,userId,movieId,rating,timestamp
0,0,0,2.5,1260759144
1,0,1,3.0,1260759179
2,0,2,3.0,1260759182
3,0,3,2.0,1260759185
4,0,4,4.0,1260759205


In [11]:
user_min, user_max, movie_min, movie_max = (ratings.userId.min(), 
    ratings.userId.max(), ratings.movieId.min(), ratings.movieId.max())
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [12]:
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
n_users, n_movies

(671, 9066)

In [13]:
#Latent factor in each embedding
n_factors = 50

In [14]:
np.randomseed=42

In [15]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

## Basic dot product model - Using Keras functional 

In [16]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(movie_in)

In [22]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
# Model(input, output)
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
/opt/anaconda/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [23]:
 model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=10,
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79835 samples, validate on 20169 samples
Epoch 1/10
79835/79835 [==============================] - 1s - loss: 2.7175 - val_loss: 3.1131
Epoch 2/10
79835/79835 [==============================] - 1s - loss: 2.2804 - val_loss: 2.9719
Epoch 3/10
79835/79835 [==============================] - 1s - loss: 2.0760 - val_loss: 2.9213
Epoch 4/10
79835/79835 [==============================] - 1s - loss: 1.9503 - val_loss: 2.9043
Epoch 5/10
79835/79835 [==============================] - 1s - loss: 1.8813 - val_loss: 2.9001
Epoch 6/10
79835/79835 [==============================] - 1s - loss: 1.8481 - val_loss: 2.8982
Epoch 7/10
79835/79835 [==============================] - 1s - loss: 1.8286 - val_loss: 2.8943
Epoch 8/10
79835/79835 [==============================] - 1s - loss: 1.8179 - val_loss: 2.8951
Epoch 9/10
79835/79835 [==============================] - 1s - loss: 1.8101 - val_loss: 2.8920
Epoch 10/10
79835/79835 [==============================] - 1s - loss: 1.8044 - val_loss: 2.8895

## Adding Bias term

In [18]:
u

Reshape{3}.0

In [15]:
def embedding_input(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg))(inp)

In [22]:
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-6)
movie_in, m = embedding_input('movie_in', n_movies, n_factors, 1e-6)

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(671, 50, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(9066, 50, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
def create_bias(inp, n_in):
    x = Embedding(n_in, 1, input_length=1)(inp)
    return Flatten()(x)

In [24]:
ub = create_bias(user_in, n_users)
mb = create_bias(movie_in, n_movies)

In [25]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(0.001), loss='mse')

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
/opt/anaconda/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers fr

In [33]:
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, epochs=10,
        validation_data=([val.userId, val.movieId], val.rating))

Train on 79835 samples, validate on 20169 samples
Epoch 1/10
79835/79835 [==============================] - 1s - loss: 8.8641 - val_loss: 3.5246
Epoch 2/10
79835/79835 [==============================] - 1s - loss: 2.6028 - val_loss: 2.2966
Epoch 3/10
79835/79835 [==============================] - 1s - loss: 2.0017 - val_loss: 2.0957
Epoch 4/10
79835/79835 [==============================] - 1s - loss: 1.8386 - val_loss: 2.0020
Epoch 5/10
79835/79835 [==============================] - 1s - loss: 1.7414 - val_loss: 1.9308
Epoch 6/10
79835/79835 [==============================] - 1s - loss: 1.6592 - val_loss: 1.8596
Epoch 7/10
79835/79835 [==============================] - 1s - loss: 1.5840 - val_loss: 1.7908
Epoch 8/10
79835/79835 [==============================] - 1s - loss: 1.5112 - val_loss: 1.7285
Epoch 9/10
79835/79835 [==============================] - 1s - loss: 1.4419 - val_loss: 1.6667
Epoch 10/10
79835/79835 [==============================] - 1s - loss: 1.3743 - val_loss: 1.6128

In [34]:
model.optimizer.lr=0.01
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=6, 
          validation_data=([val.userId, val.movieId], val.rating))

Train on 79835 samples, validate on 20169 samples
Epoch 1/6
 8192/79835 [==>...........................] - ETA: 1s - loss: 1.2661

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


79835/79835 [==============================] - 1s - loss: 1.3092 - val_loss: 1.5550
Epoch 2/6
79835/79835 [==============================] - 1s - loss: 1.2473 - val_loss: 1.5054
Epoch 3/6
79835/79835 [==============================] - 1s - loss: 1.1863 - val_loss: 1.4553
Epoch 4/6
79835/79835 [==============================] - 1s - loss: 1.1284 - val_loss: 1.4077
Epoch 5/6
79835/79835 [==============================] - 1s - loss: 1.0740 - val_loss: 1.3666
Epoch 6/6
79835/79835 [==============================] - 1s - loss: 1.0212 - val_loss: 1.3250


In [26]:
model.optimizer.lr=0.001
model.fit([trn.userId, trn.movieId], trn.rating, batch_size=64, nb_epoch=15, 
          validation_data=([val.userId, val.movieId], val.rating))

/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 79945 samples, validate on 20059 samples
Epoch 1/15
79945/79945 [==============================] - 1s - loss: 8.8545 - val_loss: 2.9677
Epoch 2/15
79945/79945 [==============================] - 1s - loss: 1.8471 - val_loss: 1.4418
Epoch 3/15
79945/79945 [==============================] - 1s - loss: 1.0813 - val_loss: 1.1855
Epoch 4/15
79945/79945 [==============================] - 1s - loss: 0.8689 - val_loss: 1.0979
Epoch 5/15
79945/79945 [==============================] - 1s - loss: 0.7627 - val_loss: 1.0653
Epoch 6/15
79945/79945 [==============================] - 1s - loss: 0.6860 - val_loss: 1.0522
Epoch 7/15
79945/79945 [==============================] - 1s - loss: 0.6222 - val_loss: 1.0446
Epoch 8/15
79945/79945 [==============================] - 1s - loss: 0.5653 - val_loss: 1.0459
Epoch 9/15
79945/79945 [==============================] - 1s - loss: 0.5123 - val_loss: 1.0439
Epoch 10/15
79945/79945 [==============================] - 1s - loss: 0.4617 - val_loss: 1.0498

In [36]:
model.save_weights(model_path+'bias.h5')

In [21]:
model.load_weights(model_path+'bias.h5')